In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/12/c6qz0v315j1d3bkc29b5ssk80000gn/T/ipykernel_44333/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [9]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [15]:
#??

## Data Preprocessing

In [11]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [26]:
import re

def clean_html(data):
    # 1. Remove inline JavaScript/CSS: <script>...</script> and <style>...</style>
    data = re.sub(r'<script.*?>.*?</script>', '', data, flags=re.DOTALL | re.IGNORECASE)
    data = re.sub(r'<style.*?>.*?</style>', '', data, flags=re.DOTALL | re.IGNORECASE)

    # 2. Remove HTML comments
    data = re.sub(r'<!--.*?-->', '', data, flags=re.DOTALL)

    # 3. Remove remaining HTML tags
    data = re.sub(r'<[^>]+>', '', data)

    return data

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [27]:
import re

def full_clean_html(data):
    # 1. Remove inline JavaScript/CSS
    data = re.sub(r'<script.*?>.*?</script>', '', data, flags=re.DOTALL | re.IGNORECASE)
    data = re.sub(r'<style.*?>.*?</style>', '', data, flags=re.DOTALL | re.IGNORECASE)

    # 2. Remove HTML comments
    data = re.sub(r'<!--.*?-->', '', data, flags=re.DOTALL)

    # 3. Remove remaining HTML tags
    data = re.sub(r'<[^>]+>', '', data)

    # 4. Remove special characters
    data = re.sub(r'[^\w\s]', '', data)

    # 5. Remove numbers
    data = re.sub(r'\d+', '', data)

    # 6. Remove single characters (including isolated letters like "a", "b", etc.)
    data = re.sub(r'\b\w\b', '', data)

    # 7. Remove single characters from the start
    data = re.sub(r'^\w\s+', '', data)

    # 8. Substitute multiple spaces with single space
    data = re.sub(r'\s+', ' ', data)

    # 9. Remove prefixed 'b' (commonly from byte strings, e.g. b'hello')
    data = re.sub(r'^b\s*', '', data)

    # 10. Convert to lowercase
    data = data.lower()

    return data.strip()


## Now let's work on removing stopwords
Remove the stopwords.

In [28]:
import nltk
from nltk.corpus import stopwords

# Download stopwords if you haven't already
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Function to remove stopwords from a string
def remove_stopwords(text):
    words = text.split()
    return ' '.join([word for word in words if word.lower() not in stop_words])

# Apply it to the 'text' column in your DataFrame
data['text'] = data['text'].apply(remove_stopwords)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/darius/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [29]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Download necessary resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

# Function to lemmatize text
def lemmatize_text(text):
    words = nltk.word_tokenize(text)
    lemmatized = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized)

# Apply to your DataFrame column (e.g., 'text')
data['text'] = data['text'].apply(lemmatize_text)




[nltk_data] Downloading package punkt to /Users/darius/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/darius/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/darius/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [30]:
print(data)

                                                  text  label
0    DEAR SIR , STRICTLY PRIVATE BUSINESS PROPOSAL ...      1
1                                                    .      0
2    Nora -- Cheryl emailed dozen memo Haiti weeken...      0
3    Dear Sir=2FMadam=2C know proposal might surpri...      1
4                                                  fyi      0
..                                                 ...    ...
995  's latest ? sound contradictory AF decide ? Sh...      0
996  TRANSFER 36,759,000.00 MILLION POUNDS YOURACCO...      1
997               Barb call explain . back country ? H      0
998       Yang travelNot free tonite.May work tomorrow      0
999  sbwhoeopSunday February 21 2010 7:42 PMHShaunH...      0

[998 rows x 2 columns]


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [31]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Separate ham and spam messages
ham_texts = data[data['label'] == 'ham']['text']
spam_texts = data[data['label'] == 'spam']['text']

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform ham messages
ham_matrix = vectorizer.fit_transform(ham_texts)
ham_word_counts = ham_matrix.sum(axis=0).A1
ham_vocab = vectorizer.get_feature_names_out()
ham_freq = pd.Series(ham_word_counts, index=ham_vocab).sort_values(ascending=False)

# Fit and transform spam messages
vectorizer = CountVectorizer()  # re-initialize to reset vocabulary
spam_matrix = vectorizer.fit_transform(spam_texts)
spam_word_counts = spam_matrix.sum(axis=0).A1
spam_vocab = vectorizer.get_feature_names_out()
spam_freq = pd.Series(spam_word_counts, index=spam_vocab).sort_values(ascending=False)

# Display top 10 words
print("Top 10 Ham Words:\n", ham_freq.head(10))
print("\nTop 10 Spam Words:\n", spam_freq.head(10))


ValueError: empty vocabulary; perhaps the documents only contain stop words

## Extra features

In [32]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

NameError: name 'data_train' is not defined

## How would work the Bag of Words with Count Vectorizer concept?

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data)

print(vectorizer.get_feature_names_out())
print(X.toarray())


['label' 'text']
[[0 1]
 [1 0]]


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Load the vectorizer
tfidf_vectorizer = TfidfVectorizer()

# 2. Vectorize the dataset (fit and transform the text)
tfidf_matrix = tfidf_vectorizer.fit_transform(data['text'])

# 3. Print the shape of the vectorized dataset
print("TF-IDF Matrix Shape:", tfidf_matrix.shape)


TF-IDF Matrix Shape: (998, 25152)


## And the Train a Classifier?

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# 1. Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    data['text'], data['label'], test_size=0.2, random_state=42)

# 2. Vectorize text with TF-IDF (fit on train, transform on test)
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# 3. Initialize and train classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)

# 4. Predict on test set
y_pred = clf.predict(X_test_tfidf)

# 5. Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.98

Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98       119
           1       1.00      0.95      0.97        81

    accuracy                           0.98       200
   macro avg       0.98      0.98      0.98       200
weighted avg       0.98      0.98      0.98       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code